## Creating the base

In [4]:
import requests as re
import logging
import json

In [14]:
# Logging to help find problems and tests
logging.basicConfig(filename = 'log.txt', format = '%(asctime)s - %(levelname)s: %(message)s'\
                    , level = logging.INFO)

In [15]:
# Pattern to the api search construction
wanted_types = ["tafsir","translation","quran","transliteration","versebyverse"]

URL = "http://api.alquran.cloud/v1/"
topic = 'edition'
lang = 'en'
format_ = 'text'

In [16]:
for t in wanted_types:
    response = re.get(url=f'{URL}{topic}?language={lang}&format={format_}&type={t}') # Filters taken from this sentence (Raw JSON files containing the Quran text in English translation with the following JSON structure:)
    print(response.text)
    
# Checked on database and looks like, tree of the wanted formats, doesn't have a english version on text format 

{"code":404,"status":"NOT FOUND","data":"Invalid format."}
{"code":200,"status":"OK","data":[{"identifier":"en.ahmedali","language":"en","name":"Ahmed Ali","englishName":"Ahmed Ali","format":"text","type":"translation","direction":"ltr"},{"identifier":"en.ahmedraza","language":"en","name":"Ahmed Raza Khan","englishName":"Ahmed Raza Khan","format":"text","type":"translation","direction":"ltr"},{"identifier":"en.arberry","language":"en","name":"Arberry","englishName":"A. J. Arberry","format":"text","type":"translation","direction":"ltr"},{"identifier":"en.asad","language":"en","name":"Asad","englishName":"Muhammad Asad","format":"text","type":"translation","direction":"ltr"},{"identifier":"en.daryabadi","language":"en","name":"Daryabadi","englishName":"Abdul Majid Daryabadi","format":"text","type":"translation","direction":"ltr"},{"identifier":"en.hilali","language":"en","name":"Hilali & Khan","englishName":"Muhammad Taqi-ud-Din al-Hilali and Muhammad Muhsin Khan","format":"text","type":

In [17]:
# Get the data in any situation
editions = {'data':[]}
for t in wanted_types:
    response = re.get(url=f'{URL}{topic}?language={lang}&format={format_}&type={t}')
    
    if response.status_code == 200:
        api_editions = response.json()
        editions['data'].append({'editions':api_editions['data'][0]['identifier']}) 
    else:
        logging.info(f"Don't have edition to TYPE {t.upper()}")


# response = re.get(url=f'{URL}{topic}?language={lang}&format={format_}&type={t}')
# response.json()

In [18]:
# print(editions)

{'data': [{'editions': 'en.ahmedali'}, {'editions': 'en.transliteration'}]}


In [ ]:
# This thing create the json, that will be used to save the data 
final_json = {'data':[]}
topic = "juz"
for editions_ in editions['data']:
    for e in editions_.values():
        for juz in range(1, 31):    
            logging.info(f"In progress juz: {juz} - e: {e}" )
            response = re.get(f"{URL}{topic}/{juz}/{e}")
            mid_level = response.json()
            for k,v in zip(mid_level["data"]["ayahs"],mid_level["data"]["ayahs"]):    
                final_json["data"].append({
                e: {
                    juz: {
                        k["surah"]["englishName"]
                        :
                        v["text"]
                        }
                    }
                })



# response = re.get(f"{URL}{topic}/1/en.ahmedali")
# t = response.json()
# for value in t['data']['ayahs']:
#     print(value['text'])
final_json

In [20]:
result = json.dumps(final_json)

In [21]:
# Creating the result file
with open('raw_json.json','w') as file_:
    file_.write(result)

## SQL

In [9]:
# Module to connect to the MySQL (Explain why not used MariaDB on the final)

import mysql.connector as mconn

logging.info("Starting connection to MySQL")

try:
    conn = mconn.connect(host='localhost',user='root',\
    password='12345',database='base',port = 3310)
    logging.info("Connected to MySQL")
except Exception as ex:
    logging.error("Failed to connect to MySQL: %s",ex)
    sys.exit()
finally:
    logging.info("Connection module finalized!")
cursor = conn.cursor()
logging.info("Cursor initialized")

In [ ]:
# cursor.execute('DROP TABLE TB_quaron;')
# conn.commit()

In [7]:
# Reading the json data
with open('result.json','r') as file_:
    raw_json = json.load(file_)
    
# Query to create table on the database
create_sql = """
                CREATE TABLE IF NOT EXISTS TB_quran (
                id integer NOT NULL PRIMARY KEY AUTO_INCREMENT,
                edition TEXT NOT NULL,
                juz TEXT NOT NULL,
                surah TEXT NOT NULL,
                text TEXT NOT NULL
                );
""" 
cursor.execute(create_sql)
conn.commit()

In [8]:
# Getting the data from json structure and Inserting under the database

for value in raw_json['data']:
    for edition,value in value.items():
        for juz,value in value.items():
            for surah,text in value.items():
                inset_sql = f"""
                    INSERT INTO TB_quran(edition,juz,surah,text) VALUES (%s,%s,%s,%s);
                """
                cursor.execute(inset_sql,tuple((edition,juz,surah,text)))
conn.commit()

## Optimization and Beyond

This archive is a Jupyter notebook and can't go to the Production, but, how that's a challenge that I think you want to see my way to work, I preferred to do here. <br/>
Now, why not use the MariaDB, I hadn`t a lot of time to do this test and I had some problems with the mariadb connector, so I quit MariaDB and did that on MySQL (That, let's agree, is almost the same thing haha), anyway the codes here can be easily used on MariaDB too.

About the Optimization:
 > All this code, can be a modularized script, that can be useful for a lot of other things.
 
 > I used raw python, but with sqlalchemy and maybe some frameworks, the job can be more easy to do.

 > All the process can be faster with more time of planning, that can be a good thing if hypothetically someone need to handle with this code on the future.

 > The errors management, can be better too, maybe more information on logs (We ever can be better..)